Trying Spark on this data.

(1) 
https://www.kaggle.com/kkhandekar/apache-spark-beginner-tutorial

(2) 
https://www.kaggle.com/tylerx/machine-learning-with-spark

(3) 
https://www.kaggle.com/anninasimon/pyspark-basics-for-beginners-part-1

https://www.kaggle.com/anninasimon/pyspark-basics-part-2

(4)
https://www.kaggle.com/nilaychauhan/pyspark-tutorial-for-beginners



### Trying (4)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telecom-churn-datasets/churn-bigml-20.csv
/kaggle/input/telecom-churn-datasets/churn-bigml-80.csv


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import datetime


In [3]:
#install Apache Spark
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s               
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=480736a49fe10d92afb41984c90afe99ec611166521c9fcb1e619ec9a3df8d27
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
#Apache Spark Libraries
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

#Apache Spark ML CLassifier Libraries
from pyspark.ml.classification import DecisionTreeClassifier,RandomForestClassifier,NaiveBayes


#Apache Spark Evaluation Library
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#Apache Spark Features libraries
from pyspark.ml.feature import StandardScaler,StringIndexer, VectorAssembler, VectorIndexer, OneHotEncoder

#Apache Spark Pipelin Library
from pyspark.ml import Pipeline

# Apache Spark `DenseVector`
from pyspark.ml.linalg import DenseVector

#Data Split Libraries
import sklearn
from sklearn.model_selection import train_test_split


#Tabulating Data
from tabulate import tabulate

#Garbage
import gc

In [5]:
!java -version

openjdk version "11.0.13" 2021-10-19
OpenJDK Runtime Environment (build 11.0.13+8-Ubuntu-0ubuntu1.20.04)
OpenJDK 64-Bit Server VM (build 11.0.13+8-Ubuntu-0ubuntu1.20.04, mixed mode, sharing)


In [6]:
sc = pyspark.SparkContext(appName="intro_to_spark")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/17 18:49:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
log_of_songs = [
        "Despacito",
        "Nice for what",
        "No tears left to cry",
        "Despacito",
        "Havana",
        "In my feelings",
        "Nice for what",
        "despacito",
        "All the stars"
]

# parallelize the log_of_songs to use with Spark
distributed_song_log = sc.parallelize(log_of_songs)

In [8]:
distributed_song_log

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [9]:
distributed_song_log

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [10]:
distributed_song_log.collect()

['Despacito',
 'Nice for what',
 'No tears left to cry',
 'Despacito',
 'Havana',
 'In my feelings',
 'Nice for what',
 'despacito',
 'All the stars']

In [11]:
def convert_song_to_lowercase(song):
    return song.lower()

In [12]:
distributed_song_log.map(convert_song_to_lowercase).collect()

['despacito',
 'nice for what',
 'no tears left to cry',
 'despacito',
 'havana',
 'in my feelings',
 'nice for what',
 'despacito',
 'all the stars']

In [13]:
spark = SparkSession \
    .builder \
    .appName("intro_to_spark") \
    .getOrCreate()

In [14]:
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1647542968734'),
 ('spark.driver.port', '44035'),
 ('spark.app.name', 'intro_to_spark'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.startTime', '1647542966880'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '2aba1b7764c6'),
 ('spark.sql.warehouse.dir', 'file:/kaggle/working/spark-warehouse'),
 ('spark.ui.showConsoleProgress', 'true')]

In [15]:
spark

In [16]:
path = "../input/telecom-churn-datasets/churn-bigml-80.csv"
data = spark.read.csv(path)

In [17]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)



In [18]:
data.describe()

DataFrame[summary: string, _c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string]

In [19]:
data.show(n=1)

+-----+--------------+---------+------------------+---------------+--------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+--------------------+-----+
|  _c0|           _c1|      _c2|               _c3|            _c4|                 _c5|              _c6|            _c7|             _c8|              _c9|           _c10|            _c11|               _c12|             _c13|              _c14|              _c15|            _c16|             _c17|                _c18| _c19|
+-----+--------------+---------+------------------+---------------+--------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+--------------------+-----+
|State|Accoun

In [20]:
data.take(5)

[Row(_c0='State', _c1='Account length', _c2='Area code', _c3='International plan', _c4='Voice mail plan', _c5='Number vmail messages', _c6='Total day minutes', _c7='Total day calls', _c8='Total day charge', _c9='Total eve minutes', _c10='Total eve calls', _c11='Total eve charge', _c12='Total night minutes', _c13='Total night calls', _c14='Total night charge', _c15='Total intl minutes', _c16='Total intl calls', _c17='Total intl charge', _c18='Customer service calls', _c19='Churn'),
 Row(_c0='KS', _c1='128', _c2='415', _c3='No', _c4='Yes', _c5='25', _c6='265.1', _c7='110', _c8='45.07', _c9='197.4', _c10='99', _c11='16.78', _c12='244.7', _c13='91', _c14='11.01', _c15='10.0', _c16='3', _c17='2.7', _c18='1', _c19='False'),
 Row(_c0='OH', _c1='107', _c2='415', _c3='No', _c4='Yes', _c5='26', _c6='161.6', _c7='123', _c8='27.47', _c9='195.5', _c10='103', _c11='16.62', _c12='254.4', _c13='103', _c14='11.45', _c15='13.7', _c16='3', _c17='3.7', _c18='1', _c19='False'),
 Row(_c0='NJ', _c1='137', _c

In [21]:
data.select('_c0').show()

+-----+
|  _c0|
+-----+
|State|
|   KS|
|   OH|
|   NJ|
|   OH|
|   OK|
|   AL|
|   MA|
|   MO|
|   WV|
|   RI|
|   IA|
|   MT|
|   IA|
|   ID|
|   VT|
|   VA|
|   TX|
|   FL|
|   CO|
+-----+
only showing top 20 rows



In [22]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime


In [23]:
data.count()

2667

In [24]:
data.select('_c0').dropDuplicates().sort('_c0').show()

+---+
|_c0|
+---+
| AK|
| AL|
| AR|
| AZ|
| CA|
| CO|
| CT|
| DC|
| DE|
| FL|
| GA|
| HI|
| IA|
| ID|
| IL|
| IN|
| KS|
| KY|
| LA|
| MA|
+---+
only showing top 20 rows



In [25]:
data.select(['_c1', '_c2']).where(data._c0 == 'CA').collect()

[Row(_c1='116', _c2='415'),
 Row(_c1='151', _c2='415'),
 Row(_c1='93', _c2='415'),
 Row(_c1='96', _c2='510'),
 Row(_c1='113', _c2='415'),
 Row(_c1='112', _c2='415'),
 Row(_c1='60', _c2='415'),
 Row(_c1='72', _c2='408'),
 Row(_c1='92', _c2='408'),
 Row(_c1='103', _c2='415'),
 Row(_c1='37', _c2='415'),
 Row(_c1='77', _c2='510'),
 Row(_c1='138', _c2='510'),
 Row(_c1='120', _c2='510'),
 Row(_c1='137', _c2='415'),
 Row(_c1='95', _c2='415'),
 Row(_c1='105', _c2='415'),
 Row(_c1='120', _c2='415'),
 Row(_c1='33', _c2='408'),
 Row(_c1='112', _c2='415'),
 Row(_c1='75', _c2='415'),
 Row(_c1='87', _c2='415'),
 Row(_c1='185', _c2='408'),
 Row(_c1='82', _c2='415')]